In [ ]:
import os
os.chdir('..')

from util import CONFIG
CONFIG.set_use_mpl_tables(True)

## Section 9.1

#### Section 9.1.1

In [ ]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [ ]:
from util import print_state_table

print_state_table(state)

In [ ]:
predicate = lambda k: True if k == 3 else False

In [ ]:
def oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1
            
oracle(state, predicate)

In [ ]:
print_state_table(state)

#### Section 9.1.2

Listing 9.1

In [ ]:
from util import inner

def inversion(original, current):
    proj = inner(original, current)
    for k in range(len(current)):
        current[k] = 2*proj*original[k] - current[k]

In [ ]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy() #<1>

oracle(state, predicate)

In [ ]:
inversion(s, state) #<1>

In [ ]:
print_state_table(state)

In [ ]:
from util import generate_state

n = 3
state = generate_state(n)

In [ ]:
print_state_table(state)

In [ ]:
s = state.copy()

predicate = lambda k: True if k == 5 else False
oracle(state, predicate)

In [ ]:
print_state_table(state)

In [ ]:
inversion(s, state)

In [ ]:
print_state_table(state)

In [ ]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy()

predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

In [ ]:
from util import is_close

amplitude_mean = sum(state)/2**n

proj = inner(s, state)
for k in range(len(state)):
    if k != 3:
        assert is_close(proj*state[k], amplitude_mean)

In [ ]:
for k in range(len(state)):
    state[k] = 2*amplitude_mean-state[k]

In [ ]:
print_state_table(state)

#### Section 9.1.3

Listing 9.2

In [ ]:
from math import cos

def grover_sim(state, predicate, iterations):
    s = state.copy()

    p = sum([abs(s[k])**2 for k in items]) # <1>
    theta = asin(sqrt(p)) # <1>
    assert is_close(inner(s, state), 1)

    for it in range(1, iterations + 1):
        oracle(state, predicate)
        inversion(s, state)
        assert is_close(inner(s, state), cos(2*it*theta)) # <2>

        p = sum([abs(state[k])**2 for k in items]) # <3>
        assert is_close(p, sin((2*it + 1)*theta)**2) # <4>

In [ ]:
from math import sin, asin

def target_amplitude_uniform(n, l, j):
    theta = asin(sqrt(l/2**n))
    return sin((2*j+1)*theta)/sqrt(l)

In [ ]:
from util import is_close

n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 1)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 1))

In [ ]:
print_state_table(state)

In [ ]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 2)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 2))

In [ ]:
print_state_table(state)

In [ ]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

grover_sim(state, predicate, iterations = 3)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 3))

In [ ]:
print_state_table(state)

In [ ]:
from math import floor, pi

num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

In [ ]:
from util import generate_state

n = 3
items = [0, 1]
predicate = lambda i: True if i in items else False
iterations = int(floor(pi/4*sqrt(2**n/len(items))))

state = generate_state(n)
grover_sim(state, predicate, iterations)

In [ ]:
print_state_table(state)

#### Section 9.1.4

In [ ]:
import numpy as np

def random_transformation(n):
    import scipy.stats
    U = scipy.stats.unitary_group.rvs(2**n)

    def f_direct(state):
        assert(len(state) == 2**n)
        s = U @ state
        for k in range(len(s)):
            state[k] = s[k]

    def f_inverse(state):
        assert(len(state) == 2**n)
        s = np.conj(U.transpose()) @ state
        for k in range(len(s)):
            state[k] = s[k]

    return (f_direct, f_inverse)

In [ ]:
n = 3
f = random_transformation(n)

In [ ]:
from sim_core import init_state

state = init_state(n)
f[0](state)

In [ ]:
print_state_table(state)

Listing 9.3

In [ ]:
from math import log2

def inversion_0_transformation(f, state):
    n = int(log2(len(state)))

    transform = f[0]
    inverse_transform = f[1]

    inverse_transform(state)
    inversion(init_state(n), state)
    transform(state)

In [ ]:
predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

In [ ]:
print_state_table(state)

In [ ]:
inversion_0_transformation(f, state)

In [ ]:
print_state_table(state)

## Section 9.2

#### Section 9.2.1

Listing 9.4

In [ ]:
from sim_circuit import QuantumRegister, QuantumCircuit


def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

#### Section 9.2.2

Listing 9.5

In [ ]:
def inversion_0_circuit(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.x(q[i])

    qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

    for i in range(n):
        qc.x(q[i])

    return qc

Listing 9.6

In [ ]:
def inversion_circuit(A):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A.inverse(), q)

    qc.append(inversion_0_circuit(n), q)

    qc.append(A, q)

    return qc

#### Section 9.2.3

Listing 9.7

In [ ]:
def grover_iterate_circuit(A, O):
    n = sum(O.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(O, q)

    qc.append(inversion_circuit(A), q)

    return qc

#### Section 9.2.4

Listing 9.8

In [ ]:
def grover_circuit(A, O, iterations):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    
    qc.append(A, q)

    for i in range(1, iterations + 1):
        qc.append(grover_iterate_circuit(A, O), q)
        qc.report(f'iteration_{i}')

    return qc

In [ ]:
def prepare_uniform(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.h(q[i])

    return qc

In [ ]:
n = 3
items = [1, 3, 7]
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

qc = grover_circuit(prepare_uniform(n), phase_oracle_match(n, items), num_iterations)

In [ ]:
from util_qiskit import draw_circuit

draw_circuit(qc)

In [ ]:
for i in range(1, num_iterations + 1):
    for m in items:
        assert is_close(qc.reports[f'iteration_{i}'][2][m], (-1)**i * target_amplitude_uniform(n, len(items), i))

In [ ]:
print_state_table(qc.run())

In [ ]:
def unitary_to_circuit(U):
    assert(U.shape[0] == U.shape[1])
    m = int(log2(U.shape[0]))

    q = QuantumRegister(m)
    qc = QuantumCircuit(q)
    qc.append_u(U, q)

    return qc

In [ ]:
import scipy.stats
# import numpy as np
# np.random.rseed(seed=233423)

def random_unitary_and_circuit(m):
    A = scipy.stats.unitary_group.rvs(2**m)
    return A, unitary_to_circuit(A)

def random_circuit(m):
    return random_unitary_and_circuit(m)[1]

In [ ]:
n = 3
items = [0, 1]
iterations = 3

q = QuantumRegister(n)
qc = QuantumCircuit(q)

O = phase_oracle_match(n, items)
C = random_circuit(n)
G = grover_iterate_circuit(C, O)

qc.append(C, q)
qc.report('original')
for it in range(1, iterations + 1):
    qc.append(G, q)
    qc.report(f'iteration_{it}')

state = qc.run()
print_state_table(state)

original = qc.reports['original'][2]
prob_good = sum([abs(original[k])**2 for k in items])

theta = asin(sqrt(prob_good))
for it in range(1, iterations + 1):
    s = qc.reports[f'iteration_{it}'][2]
    assert is_close(abs(cos(2*it*theta)), abs(inner(original, s)))
    
    p = sum([abs(s[k])**2 for k in items])
    assert is_close(p, sin((2*it + 1)*theta)**2)

## Section 9.3

#### Section 9.3.1

Listing 9.9

In [ ]:
def amplitude_estimation_circuit(n, A, O, swap=True):
    c = QuantumRegister(n)
    q = QuantumRegister(sum(A.regs))
    qc = QuantumCircuit(c, q)

    for i in range(n):
        qc.h(c[i])

    qc.append(A, q)

    for i in range(n):
        for _ in range(2**i):
            if swap:
                qc.c_append(grover_iterate_circuit(A, O), c[i], q) # <1>
            else:
                qc.c_append(grover_iterate_circuit(A, O), c[n-1-i], q) # <1>

    qc.iqft(c if swap else c[::-1], swap)

    return qc

#### Section 9.3.2

In [ ]:
from math import pi

n = 5
m = 3
items = [0, 1, 2]
qc = amplitude_estimation_circuit(n, prepare_uniform(m), phase_oracle_match(m, items), False)

state = qc.run()

In [ ]:
from util import list_to_dict, plot_bars

probs = [0 for _ in range(2**n)]
for k in range(2**m): # prefix
    for j in range(2**n): # suffix
        probs[j] += abs(state[k*2**n + j])**2

plot_bars(list_to_dict(probs, True), '', 'Outcomes', 'Probability')

In [ ]:
probs_half = [2*probs[k] for k in range(len(probs)//2 + 1, len(probs))]
probs_half = [1 - sum(probs_half)] + probs_half

plot_bars(list_to_dict(probs_half, False), '', '', '')

In [ ]:
import numpy as np

v = np.argmax(probs[int(len(probs)/2):])
print('v:', v)
count = int(2**m*sin(7*pi/2**n)**2)
print('count:', count)
assert(count == len(items))

#### Section 9.3.3

In [ ]:
m = 3

np.random.seed(seed=50)
C = random_circuit(m)

print_state_table(C.run())

In [ ]:
A, C = random_unitary_and_circuit(m)

In [ ]:
def good_probs(qc, items):
    state = qc.run()
    return sum([abs(state[k])**2 for k in items])

In [ ]:
# probability of good states

# items = [0, 1]
items = range(2**m)[:2**(m-1)]
prob_g = good_probs(C, items)
print(prob_g)

In [ ]:
# Grover operator

def oracle_unitary(n, items):
    O = np.eye(2**n)
    for i in range(2**n):
        if i in items:
            O[i, i] = -1

    return O


def inversion_0_unitary(n):
    M0 = np.eye(2**n)
    M0[0, 0] = -1

    return M0

M0 = inversion_0_unitary(m)
O = oracle_unitary(m, items)

G = A @ M0 @ np.conj(A.transpose()) @ O

In [ ]:
# the squared cos/sin of half of a non-trivial eigenphase of G gives the probability of good outcomes

eigvals, eigvecs = np.linalg.eig(G)

print('eigvals', [np.round(e, 4) for e in eigvals])

eig1 = cos(np.angle(eigvals[0])/2)**2
eig2 = (eigvals[0]**0.5).real**2


print(prob_g, eig1, eig2)

if is_close(eig1, 0):
    print('using eigvals[1]')
    eig1 = cos(np.angle(eigvals[1])/2)**2
    eig2 = (eigvals[1]**0.5).real**2


    print(prob_g, eig1, eig2)


In [ ]:
assert is_close(eig1, prob_g)
assert is_close(eig2, prob_g)

In [ ]:
n = 8 # 9

C = unitary_to_circuit(A)

qc = amplitude_estimation_circuit(n, C, phase_oracle_match(m, items), swap=True)
state = qc.run()

probs = [0 for _ in range(2**n)]
for k in range(2**m): # prefix
    for j in range(2**n): # suffix
        probs[j] += abs(state[k*2**n + j])**2

# plot_bars(list_to_dict(probs, False), '', 'Outcomes', 'Probability')

In [ ]:
# sines = {}
# for k, v in enumerate(probs):
#     # key = 2**tgt_bits*np.round(np.cos(np.pi*k/2**ctrl_bits)**2, 4)
#     key = 2**m*round(cos(k*pi/2**n)**2, 4)
#     sines[key] = sines.get(key, 0) + round(v, 4)
#         
# est = max(sines, key=sines.get)
# print('est ~ ', est/2**m)

v = np.argmax(probs[int(len(probs)/2):])
estimate = round(sin(v*pi/2**n)**2, 4)
print('estimate ~ ', estimate)

assert(abs(prob_g - estimate) < 0.01) # 8-9 qubits are typically enough to get 2 decimals, but may occasionally fail